# Tutorial: Claude Code Agents + Subagents for dspy.RLM via fleet-rlm

Audience:
- Developers setting up Claude Code agent workflows around `dspy.RLM` with `fleet-rlm`.

Prerequisites:
- `uv` installed
- Modal account access
- Local checkout of this repository

Learning goals:
- Configure env, Modal auth/secrets, and scaffold assets
- Understand Claude agent/subagent roles used by fleet-rlm
- Verify setup with CLI and local filesystem checks


## Outline

1. Setup constants and safe local checks
2. Install dependencies and configure `.env`
3. Authenticate Modal and create secrets/volume
4. Install fleet-rlm scaffold assets (`init`)
5. Enable Claude Code teams + map agent topology
6. Verify runtime with CLI and interactive commands
7. Troubleshoot common setup failures


In [1]:
from __future__ import annotations

from pathlib import Path
import json

CLAUDE_HOME = Path.home() / '.claude'
EXPECTED = {
    'skills': CLAUDE_HOME / 'skills',
    'agents': CLAUDE_HOME / 'agents',
    'teams': CLAUDE_HOME / 'teams',
    'hooks': CLAUDE_HOME / 'hooks',
}

status = {
    key: {
        'path': str(path),
        'exists': path.exists(),
    }
    for key, path in EXPECTED.items()
}
print(json.dumps(status, indent=2))


{
  "skills": {
    "path": "/Users/zocho/.claude/skills",
    "exists": true
  },
  "agents": {
    "path": "/Users/zocho/.claude/agents",
    "exists": true
  },
  "teams": {
    "path": "/Users/zocho/.claude/teams",
    "exists": true
  },
  "hooks": {
    "path": "/Users/zocho/.claude/hooks",
    "exists": true
  }
}


## Step 1 - Environment and Package Install

Run these commands from repo root:

```bash
uv sync --extra dev --extra interactive
# optional full surface
uv sync --extra dev --extra interactive --extra mcp --extra server
cp .env.example .env
```

Required `.env` keys:
- `DSPY_LM_MODEL`
- `DSPY_LLM_API_KEY`

Optional:
- `DSPY_LM_API_BASE`
- `DSPY_LM_MAX_TOKENS`



## Step 2 - Modal Auth + Secrets + Volume

```bash
uv run modal setup
uv run modal secret create LITELLM \
  DSPY_LM_MODEL=... \
  DSPY_LLM_API_KEY=... \
  DSPY_LM_API_BASE=... \
  DSPY_LM_MAX_TOKENS=...

# optional persistence
uv run modal volume create rlm-volume-dspy
```

Then verify:

```bash
uv run fleet-rlm check-secret
```



## Step 3 - Install Scaffold Assets for Claude Code

```bash
uv run fleet-rlm init --list
uv run fleet-rlm init
```

This installs skills/agents/teams/hooks into `~/.claude/`.



In [2]:
# Optional verification cell (non-destructive)
from pathlib import Path

claude_home = Path.home() / '.claude'
for rel in ['skills', 'agents', 'teams', 'hooks']:
    p = claude_home / rel
    print(f'{rel:>6}:', 'OK' if p.exists() else 'MISSING', '-', p)

sample_paths = [
    claude_home / 'agents' / 'rlm-orchestrator.md',
    claude_home / 'agents' / 'rlm-specialist.md',
    claude_home / 'agents' / 'rlm-subcall.md',
    claude_home / 'agents' / 'modal-interpreter-agent.md',
]
for p in sample_paths:
    print('agent file:', 'OK' if p.exists() else 'MISSING', '-', p.name)


skills: OK - /Users/zocho/.claude/skills
agents: OK - /Users/zocho/.claude/agents
 teams: OK - /Users/zocho/.claude/teams
 hooks: OK - /Users/zocho/.claude/hooks
agent file: OK - rlm-orchestrator.md
agent file: OK - rlm-specialist.md
agent file: OK - rlm-subcall.md
agent file: OK - modal-interpreter-agent.md


## Step 4 - Enable Claude Code Agent Teams

Set this in your Claude Code settings or shell env:

```bash
export CLAUDE_CODE_EXPERIMENTAL_AGENT_TEAMS=1
```

Team config and inbox assets are installed under:
- `~/.claude/teams/{team}/config.json`
- `~/.claude/teams/{team}/inboxes/*`



## Step 5 - Agent / Subagent Topology for dspy.RLM

Recommended role map:
- `rlm-orchestrator`: high-level task decomposition and routing
- `rlm-specialist`: complex long-context RLM execution
- `rlm-subcall`: lightweight delegated sub-LLM calls
- `modal-interpreter-agent`: direct sandbox execution and diagnostics

Example invocation ideas:
- `@rlm-orchestrator Analyze these docs and produce a migration plan`
- `@rlm-specialist Extract API endpoints from this large codebase`
- `Task(agent="rlm-subcall", prompt="Summarize section 3")`



## Step 6 - Verify Runtime with CLI

```bash
uv run fleet-rlm check-secret
uv run fleet-rlm code-chat
# legacy fallback
uv run fleet-rlm run-react-chat --legacy
```

In interactive sessions, useful commands include:
- `/tools`
- `/load <path>`
- `/trace compact|verbose|off`



## Exercise

Pick one real task and map it to orchestrator + specialist + subcall responsibilities.
Then define concrete prompts for each role.



In [3]:
# Exercise answer scaffold
plan = {
    'task': 'Replace with your real dspy.RLM workflow task',
    'roles': {
        'rlm-orchestrator': 'Break work into staged objectives and assign specialist calls',
        'rlm-specialist': 'Execute deep RLM analysis on primary artifacts',
        'rlm-subcall': 'Run focused sub-queries for chunk-level summarization',
        'modal-interpreter-agent': 'Handle sandbox execution troubleshooting when needed',
    },
    'prompts': {
        'orchestrator_prompt': '',
        'specialist_prompt': '',
        'subcall_prompt': '',
    },
}
print(json.dumps(plan, indent=2))


{
  "task": "Replace with your real dspy.RLM workflow task",
  "roles": {
    "rlm-orchestrator": "Break work into staged objectives and assign specialist calls",
    "rlm-specialist": "Execute deep RLM analysis on primary artifacts",
    "rlm-subcall": "Run focused sub-queries for chunk-level summarization",
    "modal-interpreter-agent": "Handle sandbox execution troubleshooting when needed"
  },
  "prompts": {
    "orchestrator_prompt": "",
    "specialist_prompt": "",
    "subcall_prompt": ""
  }
}


## Common Failure Modes

- **Modal auth errors**: run `uv run modal setup` again.
- **Missing secret keys**: re-create `LITELLM` secret with all required vars.
- **Scaffold missing files**: run `uv run fleet-rlm init --force`.
- **Teams not visible**: ensure `CLAUDE_CODE_EXPERIMENTAL_AGENT_TEAMS=1` is enabled.
- **Interactive deps missing**: run `uv sync --extra dev --extra interactive`.

